In [1]:
import os
import sys
import numpy as np
import cv2
#sys.path.append(os.path.dirname(__file__) + "/../")
from scipy.misc import imread, imsave
from skimage.measure import structural_similarity as ssim
from config import load_config
from dataset.factory import create as create_dataset
from nnet import predict
from util import visualize
import cv2
from dataset.pose_dataset import data_to_input


from multiperson.detections import extract_detections
from multiperson.predict import SpatialModel, eval_graph, get_person_conf_multicut
from multiperson.visualize import PersonDraw, visualize_detections
import matplotlib.pyplot as plt
import tensorflow as tf


# In[10]:


def mse(imageA,imageB):
    err = np.sum((imageA.astype("float")-imageB.astype("float"))**2)
    err /= float(imageA.shape[0]*imageA.shape[1])
    return err

def compare_images(imageA, imageB, title):
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB)
    fig = plt.figure(title)
    plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))

    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(imageA, cmap = plt.cm.gray)
    plt.axis("off")
    ax = fig.add_subplot(1, 2, 2)
    
    plt.imshow(imageB, cmap = plt.cm.gray)
    plt.axis("off")
    plt.show()
    
    return(s,m)

In [6]:
tf.reset_default_graph()
cfg = load_config("demo/pose_cfg_multi.yaml")
dataset = create_dataset(cfg)
sm = SpatialModel(cfg)
sm.load()
draw_multi = PersonDraw()
# Load and setup CNN part detector
sess, inputs, outputs = predict.setup_pose_prediction(cfg)

# Read image from file
dir=os.listdir("stick")
k=0
cap=cv2.VideoCapture(0)
i=0
while True:
        if i%20 == 0:                   
                ret, orig_frame= cap.read()
                frame = cv2.resize(orig_frame, (0, 0), fx=0.30, fy=0.30)
                image= frame
                sse=0
                mse=0
                
                image_batch = data_to_input(frame)

                # Compute prediction with the CNN
                outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})

                scmap, locref, pairwise_diff = predict.extract_cnn_output(outputs_np, cfg, dataset.pairwise_stats)

                detections = extract_detections(cfg, scmap, locref, pairwise_diff)

                unLab, pos_array, unary_array, pwidx_array, pw_array = eval_graph(sm, detections)

                person_conf_multi = get_person_conf_multicut(sm, unLab, unary_array, pos_array)
                img = np.copy(image)
                #coor = PersonDraw.draw()
                visim_multi = img.copy()
                co1=draw_multi.draw(visim_multi, dataset, person_conf_multi)
                plt.imshow(visim_multi)
                cv2.destroyAllWindows()
                #plt.show()
                visualize.waitforbuttonpress()
                #print("this is draw : ", co1)
                if k==1:
                    qwr = np.zeros((1920,1080,3), np.uint8)

                    cv2.line(qwr, co1[5][0], co1[5][1],(255,0,0),3)
                    cv2.line(qwr, co1[7][0], co1[7][1],(255,0,0),3)
                    cv2.line(qwr, co1[6][0], co1[6][1],(255,0,0),3)
                    cv2.line(qwr, co1[4][0], co1[4][1],(255,0,0),3)

                    cv2.line(qwr, co1[9][0], co1[9][1],(255,0,0),3)
                    cv2.line(qwr, co1[11][0], co1[11][1],(255,0,0),3)
                    cv2.line(qwr, co1[8][0], co1[8][1],(255,0,0),3)
                    cv2.line(qwr, co1[10][0], co1[10][1],(255,0,0),3)
                    # In[9]:
                    cv2.imshow('r',qwr)
                    qwr2="stick/frame"+str(k)+".jpg"
                    qw1 = cv2.cvtColor(qwr, cv2.COLOR_BGR2GRAY)
                    qw2= cv2.cvtColor(qwr2, cv2.COLOR_BGR2GRAY)

                    fig = plt.figure("Images")
                    images = ("Original", qw1), ("Contrast", qw2)
                    for (i, (name, image)) in enumerate(images):
                            ax = fig.add_subplot(1, 3, i + 1)
                            ax.set_title(name)
                    plt.imshow(hash(tuple(image)))
                    # compare the images
                    s,m=compare_images(qw1, qw2, "Image1 vs Image2")
                    k+=1
                    sse=s
                    mse=m
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    print("sse score : ", sse)
                    print("Mean squared error : ", mse)
                    break
                
cap.release()
cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


num_people:  0
#tracked objects: 0
num_people:  1
#tracked objects: 0
num_people:  2
#tracked objects: 1
num_people:  3
#tracked objects: 2
num_people:  2
#tracked objects: 2
num_people:  2
#tracked objects: 2
num_people:  3
#tracked objects: 2
num_people:  3
#tracked objects: 3
num_people:  1
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  3
#tracked objects: 1
num_people:  3
#tracked objects: 3
num_people:  3
#tracked objects: 3
num_people:  3
#tracked objects: 3
num_people:  1
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  3
#tracked objects: 1
num_people:  1
#tracked objects: 1
num_people:  3
#tracked objects: 1
num_people:  4
#tracked objects: 3
num_people:  3
#tracked objects: 3
num_people:  3
#tracked objects: 3
num_people:  3
#tracked objects: 3
num_people:  3
#tracked objects: 3
num_people:  1
#tracked objects: 1
num_people:  2
#trac

KeyboardInterrupt: 

In [12]:
cap=cv2.VideoCapture('demo/seed.mp4')
i=0
j=0
cap1=cv2.VideoCapture('demo/comp.mp4')
while True:
    ret, orig_frame= cap.read()
    if i%2 == 0:
                ret, orig_frame= cap.read()
                ret2, frame2 = cap1.read()
                frame = cv2.resize(orig_frame, (0, 0), fx=0.35, fy=0.35)
                frame2 = cv2.resize(orig_frame, (0, 0), fx=0.35, fy=0.35)
                image= frame
                cv2.imwrite('written/img%d.jpg'%j, frame)
                j=j+1
                image2=frame2
                if cv2.waitKey(0):
                    break
cap.release()
cv2.destroyAllWindows()

In [30]:
# Program To Read video 
# and Extract Frames 
import cv2 

# Function to extract frames 
def FrameCapture(path): 
	
	# Path to video file 
    vidObj = cv2.VideoCapture(path) 

	# Used as counter variable 
    count = 0
    i=0

	# checks whether frames were extracted 
    success = 1
    while success: 
        if i%30==0:
            # vidObj object calls read 
            # function extract frames 
            success, image = vidObj.read() 

            # Saves the frames with frame-count 
            cv2.imwrite("written/frame%d.jpg" % count, image) 

            count += 1

# Driver Code 
	# Calling the function 
	


In [31]:
FrameCapture('demo/seed.mp4') 

In [24]:
#loop for creating stick figures for multiple images :-
for i in range ()

SyntaxError: invalid syntax (<ipython-input-24-a21be215cc0b>, line 2)

In [25]:
dir = os.listdir("written")

In [2]:
s

NameError: name 's' is not defined